In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import proplot as pplt
from scipy.fftpack import * 

## 读入数据

2022.01.08

- 因为xarray会将读入nc文件含有units = days 的数据识别为日期，decode_times 需要关闭，从而读入dtype为float64的 天数，而非一个时间dtype

In [ ]:
ds_in = {}
for mod_name in ['obs','vr', 'rcm']:
    ds_in[mod_name] = {}
    ds_in[mod_name]['am'] = xr.open_dataset("/raid52/yycheng/MPAS/REFERENCE/TEMP_DATA_large/pre/extreme/extreme_"+mod_name+"_4-5.nc", decode_times = False)
    ds_in[mod_name]['jja'] = xr.open_dataset("/raid52/yycheng/MPAS/REFERENCE/TEMP_DATA_large/pre/extreme/extreme_"+mod_name+"_6-8.nc", decode_times = False)
vars_names = list(ds_in[mod_name]['am'].variables)[3:]
vars_names

## 计算平均

In [ ]:
ds_mean = {}
for mod_name in ['obs','vr','rcm']:
    ds_mean[mod_name] = {}
    for season in ['am','jja']:
        ds_mean[mod_name][season] = ds_in[mod_name][season].mean(dim = 'time')

lon = ds_mean['obs']['am']['CDD'].lon.values
lat = ds_mean['obs']['am']['CDD'].lat.values

## 绘图部分
2022.01.09

为了方便绘图，更换了版本，用来传入控制左边标题的参数 proplot 0.6.4-py_0 --> 0.9.5-pyhd8ed1ab_1

### 绘图准备部分
包含写为函数的地图的绘制，以等距的spacing norm

In [ ]:
## 地图绘制预备数据
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.patches as mpatches
import cmaps as cmaps

In [ ]:
def border_plot(axs):
    """
    进行行政区划的绘制，通过shapefilereader绘制存档的shp文件，需要传入axs，并逐个绘制
    比较消耗时间，调整完毕后最后添加边界的绘制
    """
    ##---- 直接绘图，从边界文件添加
    # for ax_ind in axs:
    # for line in borders:
    #     axs.plot(line[0::100], line[1::100], lw = 0.5, color='gray',transform=ccrs.Geodetic())
    #     axs.plot(line[0::10], line[1::10], lw = 0.4, color='black',transform=ccrs.Geodetic())
    ##---- 使用shp文件添加
        # shapefile数据下载的位置：
    # http://gaohr.win/site/blogs/2017/2017-04-18-GIS-basic-data-of-China.html
    world_border_shapefile = "/m2data2/yycheng/data_stage/CN-border/World/country.shp"
    river_border_shapefile =  "/raid52/yycheng/MPAS/REFERENCE/MODEL_CONSTANT/R1/" + "hyd1_4l.shp"
    southsea_shapefile     = "/m2data2/yycheng/data_stage/CN-border/SouthSea/" + "southsea_island.shp"
    ninelines_shapefile     = "/m2data2/yycheng/data_stage/CN-border/SouthSea/" + "nine_lines.shp"
    ## 来源： 沛沛的诸省 + 诸岛
    bou24p_shapefile     = "/m2data2/yycheng/data_stage/CN-border/peipeihelp/" + "bou2_4p.shp"
    ## 来源： https://www.resdc.cn/data.aspx?DATAID=200
    province_shapefile     = "/m2data2/yycheng/data_stage/CN-border/CN-sheng/" + "change_proj_CN-sheng-A.shp"

    for ax in axs:
        # world     = shpreader.Reader(world_border_shapefile).geometries()
        # river     = shpreader.Reader(river_border_shapefile).geometries()
        river     = shpreader.Reader(river_border_shapefile, encoding = "gbk")
        # bou24p    = shpreader.Reader(bou24p_shapefile).geometries()
        ninelines = shpreader.Reader(ninelines_shapefile).geometries()
        province  = shpreader.Reader(province_shapefile).geometries()
        # ax.add_geometries(river, ccrs.PlateCarree(), facecolor='none', edgecolor='b', linewidth=0.4, zorder=1)
        # ax.add_geometries(world, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.4, zorder=1)
        # ax.add_geometries(bou24p, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.6, zorder=1) # 沛沛map
        ax.add_geometries(province, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.6, zorder=1) # 地资所
        ax.add_geometries(ninelines, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.6, zorder=1)
        # 绘制部分的shapefile
        for region in river.records():
            if (region.attributes['NAME'] in ['黄河','长江']):
                # print("----- draw river! -----")
                # 此处需要使用 [] 让region.geometry可以迭代
                ax.add_geometries([region.geometry], ccrs.PlateCarree(), facecolor='none', edgecolor='b', linewidth=0.4, zorder=1)


### 绘图测试部分
因为子图过多，此处测试色板 norm ticks等要素

In [ ]:
# ticks = np.linspace(0,100,11)  # CDD CWD
# ticks = np.linspace(0,10000,int(10000 / 250 + 1)) # RX5day
# ticks = np.concatenate( (np.linspace(0,4000,21),np.linspace(5000,12000,8)) , axis = 0) # RX5day
ticks =np.concatenate( (np.linspace(0,40,41),np.linspace(50,120,8)) , axis = 0)
# ticks =  np.concatenate( (np.linspace(0,40,21),np.linspace(50,100,6)) , axis = 0)

# cmap = cmaps.MPL_s3pcpn_l
cmap = cmaps.BkBlAqGrYeOrReViWh200
fig, axs = pplt.subplots(nrows = 1,ncols = 3)
var = 'CDD'
mcontourf = axs[0].contourf(ds_mean['obs']['am'][var].values, cmap = cmap, levels = ticks, norm = 'segmented')
mcontourf = axs[1].contourf(ds_mean['vr']['am'][var].values, cmap = cmap, levels = ticks, norm = 'segmented')
mcontourf = axs[2].contourf(ds_mean['rcm']['am'][var].values, cmap = cmap, levels = ticks, norm = 'segmented')
axs.format(title = var)
axs[2].colorbar(mcontourf, ticks = ticks[::2], length = 0.9, extend = 'max',title=color_bar_title[var])

### 完整绘图部分

In [ ]:
# import proplot as plot
from matplotlib import pyplot as plt
import proplot as plot
import collections
# ----- get filter vars coords-----

#----- create plot -----
fig, axs = plot.subplots(ncols=6 ,nrows=4, proj=('cyl'), share = True, wspace=(0, 0, 1, 0, 0), hspace=(0, 1, 0))
m_contour_list = [] # 用于保存contour设置，后续设置colorbar使用

#----- 添加海洋以及行政区划 -----
border_plot(axs)
    
#----- colorbar ticks 统一设置 -----
cmap_dict = {}
cmap_dict['RX5day'] = cmaps.MPL_s3pcpn_l
cmap_dict['SDII'] = cmaps.MPL_s3pcpn_l
cmap_dict['CDD'] = cmaps.BkBlAqGrYeOrReViWh200
cmap_dict['CWD'] = cmaps.BkBlAqGrYeOrReViWh200

ticks_dict = {}
ticks_dict['RX5day'] = np.concatenate( (np.linspace(0,300,31),np.linspace(300,600,11)[1:]) , axis = 0) # RX5day
# ticks_dict['RX5day'] = np.linspace(0,400,51)
ticks_dict['SDII']   = np.concatenate( (np.linspace(0,30,31),[40,50]) , axis = 0)
# ticks_dict['SDII']   = np.linspace(0,30,31)
ticks_dict['CDD']    = np.concatenate( (np.linspace(0,40,41),np.linspace(50,120,8)) , axis = 0)
ticks_dict['CWD']    = np.concatenate( (np.linspace(0,40,41),np.linspace(50,120,8)) , axis = 0)

plot_ind = 0

def nested_dict():
    return collections.defaultdict(nested_dict)
mcontourf_dict = nested_dict()

for var in vars_names:    
    for season in ['am', 'jja']:
        for mod_name in ['obs','vr','rcm']:
            
            # 使用了均分的norm，处理不等距的ticks
            mcontourf_temp = axs[plot_ind].contourf(lon, lat, ds_mean[mod_name][season][var].values,\
            cmap=cmap_dict[var], levels = ticks_dict[var], norm = 'segmented')

            # axs[plot_ind].format(rtitle = season, title = mod_name.upper(), ltitle = var)
            mcontourf_dict[season][mod_name][var] = mcontourf_temp # 合并所有绘图obj到一个deep dict之中
            
            plot_ind = plot_ind + 1

#----- add color bar-----
color_bar_title = {}
color_bar_title['RX5day'] = '5-day precipitation amount ' + r"$[mm/d]$"
color_bar_title['SDII']   = 'precipitation during wet days ' + r"$[mm/d]$"
color_bar_title['CDD']    = 'consecutive dry days ' + "[days]"
color_bar_title['CWD']    = 'consecutive wet days ' + "[days]"

plot_ind = 1
for var in vars_names:
    fig.colorbar(mcontourf_dict['am']['obs'][var], loc='r', width=0.1, extend = 'both',length = 0.96,\
    ticklabelsize=5,labelsize = 7,ticks=ticks_dict[var][::2], title=color_bar_title[var], rows = plot_ind)
    plot_ind = plot_ind + 1

# ----- format setting -----
axs.format(
abc=True,
abcloc = 'ul',
#----- 地图底图设置 -----
# reso = 'x-hi',
reso = 'med',
# coast = False,
coast = True,
coastlinewidth = 0.4,
borders = False,
lakes = False,
land  = False,
ocean = False,
# cartopyautoextent = True, 
# borderslinewidth=.5,
labels = False,
longrid  = True,
latgrid  = True,
#-----GEO axis-----
lonlim=(70, 140), latlim=(5, 60),
gridlabelsize = 5,
gridminor = True,
lonlocator = np.arange(70,142,10),
latlocator = np.arange(5,70+2,10),
lonminorlocator = np.arange(70,140+2,2),
latminorlocator = np.arange(5,70+2,2),
#-----line label-----
# linewidth = 0.5,
# suptitle="3000km-2000km bandpass 500hPa height(1998-06 timemean)",
# suptitle="precipitation",
toplabels=(' \nOBS', 'AM\nVR', ' \nRCM', ' \nOBS', 'JJA\nVR', ' \nRCM'),
leftlabels=('RX5day', 'SDII','CDD','CWD'),
rc_kw = {'leftlabel.rotation':90.}
)

# 合并子图之后控制边界的labels绘制
axs[:-1,0].format(labels = True, lonlabels = False, latlabels = True)
axs[3,1:].format(labels = True, lonlabels = True, latlabels = False)
axs[-1,0].format(labels = True, lonlabels = True, latlabels = True)
#----- save figure -----
fig.patch.set_facecolor('white')
plt.savefig('./output_pic/pre_extreme_2022.01.11.png', dpi=600, facecolor= "white")
# plt.savefig('./output_pic/hgt_idctn.png', dpi=300, facecolor= "white")